In [2]:

from multiprocessing import Pool
from tqdm import tqdm
from spacy.lang.en import English
import gensim.corpora as corpora
from gensim.models import LdaMulticore
import multiprocessing as mp
import json
import re
import numpy as np
import pandas as pd
from pprint import pprint
import pickle
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
from nltk.corpus import wordnet as wn
#nltk.download('stopwords')
#nltk.download('wordnet')
# spacy for lemmatization
import spacy
from spacy.lang.en import English
parser = English()

#Sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
# Plotting tools

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
def tokenize(doc):
    tokens = doc.split(" ")
    tokens = [word for word in tokens if len(word.strip())]
    return tokens

print("Loading tech corpus")
with open("../data/tech_review_word_corpus.pkl","rb") as f:
    tech_review_corpus = pickle.load(f)
    
reviews = pd.DataFrame(tech_review_corpus).review.tolist()
print("Tokenize the corpus")

with open(( "../data/stop_words.json"), "r") as f:
    stop_words = json.load(f)

vectorizer = CountVectorizer( min_df = 5,max_df=.90, tokenizer=tokenize, stop_words=stop_words, ngram_range=(1, 2))

np.random.seed(100)

Loading tech corpus
Tokenize the corpus


In [3]:
len(reviews)

45251

## Sampling

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(reviews,test_size=0.01, random_state=100)

In [5]:
print(len(X_train))
print(len(X_test))

44798
453


In [6]:
(X_test)

['nice to work here company environment be good i learn new thing from this company in tech mahindra work culture be good i never find the hard part of a job in this company',
 "accenture a a company be quite good however some project and people be really sick if you get into any such project that 's it end of life management be useless they only bother about their profit only if you get into project that have onsite will you be able to be a little happy",
 'it be a great place to work i learn a lot about the financial market i enjoy help support the trade room staff with analyze their portfolio use numerous analytic tool it provide me numerous opportunity for advancement until the re-organization which result in my layoff back in 1994',
 'we can work but not great place to work more stress work full work load we can not grow and learn more here we can not development ourself new thing we just work with out any satisfactory',
 'this be a great place to work grow and advance your career

In [7]:
X = vectorizer.fit_transform(X_train)
print("Total Vocab Size", len(vectorizer.vocabulary_))

Total Vocab Size 28806


In [8]:
from gensim.corpora.dictionary import Dictionary
def vect2gensim(vectorizer, dtmatrix):
     # transform sparse matrix into gensim corpus and dictionary
    corpus_vect_gensim = gensim.matutils.Sparse2Corpus(dtmatrix, documents_columns=False)
    dictionary = Dictionary.from_corpus(corpus_vect_gensim,
        id2word=dict((id, word) for word, id in vectorizer.vocabulary_.items()))

    return (corpus_vect_gensim, dictionary)
corpus, id2word = vect2gensim(vectorizer, X)

In [13]:
#final_lda_model = LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           workers = 2,
                                           passes=10,
                                           alpha="symmetric",
                                           eta=0.91,
                                           per_word_topics=True)

In [15]:
#final_lda_model.save('../results/word-k8-gensim-lda3model')

In [22]:
from gensim import  models

final_lda_model =  models.LdaModel.load('../results/word-k8-gensim-lda.model')

In [23]:
# Select the model and print the topics
optimal_model = final_lda_model
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=30))

[(0,
  '0.042*"work" + 0.027*"part" + 0.025*"enjoy" + 0.021*"job" + 0.016*"hard" + '
  '0.012*"part job" + 0.011*"learn" + 0.011*"help" + 0.011*"time" + '
  '0.010*"management" + 0.009*"hard part" + 0.009*"always" + 0.009*"typical" + '
  '0.009*"co-worker" + 0.008*"enjoy part" + 0.007*"customer" + 0.006*"enjoy '
  'work" + 0.006*"issue" + 0.005*"challenge" + 0.005*"get" + 0.005*"make" + '
  '0.004*"people" + 0.004*"typical work" + 0.004*"call" + 0.004*"need" + '
  '0.004*"fun" + 0.004*"every" + 0.004*"great" + 0.004*"project" + '
  '0.004*"manager"'),
 (1,
  '0.036*"project" + 0.029*"good" + 0.024*"salary" + 0.015*"depend" + '
  '0.014*"less" + 0.009*"hike" + 0.009*"job" + 0.008*"get" + 0.008*"onsite" + '
  '0.007*"security" + 0.007*"job security" + 0.007*"growth" + '
  '0.006*"opportunity" + 0.006*"fresher" + 0.006*"low" + 0.006*"management" + '
  '0.005*"much" + 0.005*"con" + 0.005*"depend project" + 0.005*"base" + '
  '0.005*"onsite opportunity" + 0.004*"compare" + 0.004*"bad" + 0.0

In [24]:
lda_display = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word, sort_topics=False)
pyLDAvis.display(lda_display)

In [18]:
aspect = {}
for t in range(0, optimal_model.num_topics):
    aspect['topic {}: '.format(t)] = [v[0] for v in optimal_model.show_topic(t, 50)]

In [19]:
with open("../results/gensim/word-gensim-lda-topic-k-8.json", "w") as f:
                json.dump(aspect, f)

In [20]:
with open("../data/all_reviews.pkl","rb") as f:
    reviews_ = pickle.load(f)
    
job_filter = pd.read_csv("../data/filter_job_titles.csv")

job_filters = job_filter.clean_job_title.tolist()
idx = (reviews_.language == "en")
reviews_ = reviews_.loc[idx,:]

idx = (reviews_.clean_job_title.isin(job_filters))
reviews_ = reviews_.loc[idx,:].reset_index()

print(reviews_.shape)
#display(reviews_.head())


(45251, 16)


In [96]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=reviews):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=final_lda_model, corpus=corpus, texts=X_train)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()

df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

#df_dominant_topic["Review"] = reviews_["review"] 
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,5.0,0.5206,"work, good, place, balance, life, culture, environment, great, good work, management",wonderful work culture management be very friendly and supportive get to learn a lot a the cultu...
1,1,4.0,0.4500,"work, management, people, get, employee, great, n't, job, year, manager",flexible timing you can start your work at any time complete your work and you can leave to get ...
2,2,5.0,0.7117,"work, good, place, balance, life, culture, environment, great, good work, management",cognizant be a good company if you want hike in your career early on and also it have a good job...
3,3,1.0,0.6549,"project, good, salary, depend, less, hike, job, get, onsite, security",i have be work for tata consultancy service full-time 2 5 year pros good work ethical empathetic...
4,4,3.0,0.9663,"system, development, software, support, application, work, project, use, test, client",• analysis of specification provide by client • development use c asp.net and web service • test...
5,5,3.0,0.4710,"system, development, software, support, application, work, project, use, test, client",tsys be a good company their all policy be for benefit of employee it be a product base company ...
6,6,6.0,0.3905,"learn, work, good, lot, technology, place, experience, thing, get, great",love my job of identity and access management for ibm learn a lot and ibm be a place that would ...
7,7,0.0,0.9125,"work, part, enjoy, job, hard, part job, learn, help, time, management",* typical day at work* what you learned* management* your co-workers* the hard part of the job* ...
8,8,6.0,0.4970,"learn, work, good, lot, technology, place, experience, thing, get, great",it 's a place to explore and dive into coding great team to share your view and they will give y...
9,9,5.0,0.4174,"work, good, place, balance, life, culture, environment, great, good work, management",it appreciate the effort of employee in various reward way but salary hike be not good and timel...


In [26]:
with open("tech_review_with_topics.pkl","wb") as f:
    pickle.dump(df_dominant_topic, f)

In [27]:
df_dominant_topic.groupby('Dominant_Topic').size()

Dominant_Topic
0.0     5423
1.0     2530
2.0       73
3.0     3337
4.0    11273
5.0    10622
6.0     8895
7.0     2645
dtype: int64

[['system,',
  'development,',
  'software,',
  'support,',
  'application,',
  'work',
  'project',
  'use',
  'test,',
  'client'],
 ['accenture',
  'company',
  'quite',
  'good',
  'however',
  'some',
  'project',
  'and',
  'people',
  'really',
  'sick',
  'you',
  'get',
  'into',
  'any',
  'such',
  'project',
  'that',
  'end',
  'life',
  'management',
  'useless',
  'they',
  'only',
  'bother',
  'about',
  'their',
  'profit',
  'only',
  'you',
  'get',
  'into',
  'project',
  'that',
  'have',
  'onsite',
  'will',
  'you',
  'able',
  'little',
  'happy'],
 ['great',
  'place',
  'work',
  'learn',
  'lot',
  'about',
  'the',
  'financial',
  'market',
  'enjoy',
  'help',
  'support',
  'the',
  'trade',
  'room',
  'staff',
  'with',
  'analyze',
  'their',
  'portfolio',
  'use',
  'numerous',
  'analytic',
  'tool',
  'provide',
  'numerous',
  'opportunity',
  'for',
  'advancement',
  'until',
  'the',
  're-organization',
  'which',
  'result',
  'layoff',
  

In [149]:
test_x = vectorizer.fit_transform(X_test)
test_corpus = [id2word.doc2bow(text) for text in list(map(vectorizer.build_tokenizer(),X_test))]

In [151]:

df_topic = format_topics_sentences(ldamodel=final_lda_model, corpus=test_corpus, texts=X_test)
df_topic.dropna(how='any',inplace=True)
# Format
test_df_dominant_topic = df_topic.reset_index()

test_df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


In [162]:
for i in range(10):
    print('\x1b[0;33;40m' + 'Original Review:' + '\x1b[0m')
    pprint(test_df_dominant_topic["Text"][i])
    print("Topic:",int(test_df_dominant_topic['Dominant_Topic'][i]))
    print('\x1b[0;33;40m' + "Most Ferequent words:" + '\x1b[0m')
    pprint(test_df_dominant_topic['Keywords'][i])
    print("*"*100)

Original Review:
('nice to work here company environment be good i learn new thing from this '
 'company in tech mahindra work culture be good i never find the hard part of '
 'a job in this company')
Topic: 0
Most Ferequent words:
'work, part, enjoy, job, hard, part job, learn, help, time, management'
****************************************************************************************************
Original Review:
('accenture a a company be quite good however some project and people be '
 "really sick if you get into any such project that 's it end of life "
 'management be useless they only bother about their profit only if you get '
 'into project that have onsite will you be able to be a little happy')
Topic: 4
Most Ferequent words:
"work, management, people, get, employee, great, n't, job, year, manager"
****************************************************************************************************
Original Review:
('it be a great place to work i learn a lot about the fina

### The most representative sentence for each topic¶

In [92]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf = pd.concat([sent_topics_sorteddf, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0.0,0.9873,"work, part, enjoy, job, hard, part job, learn, help, time, management",a typical day at work would be first to check the computer screen for any call that need to be r...
1,1.0,0.9687,"project, good, salary, depend, less, hike, job, get, onsite, security",good for fresher good project well onsite opportunity bad compensate a per market standard good ...
2,2.0,0.5712,"many, encourage, learn many, chance, many thing, work/life, work/life balance, mahindra, get cha...",in ibm we have many volunteer activity and club event more than that we have your learn a learni...
3,3.0,0.9960,"system, development, software, support, application, work, project, use, test, client",• involve in complete life cycle of software development include design development test and emp...
4,4.0,0.9950,"work, management, people, get, employee, great, n't, job, year, manager",my experience with hcl be not a very good one very poor communication i be offer a job full time...
5,5.0,0.9810,"work, good, place, balance, life, culture, environment, great, good work, management",good good it pretty good work culture it pretty good work life balance it pretty good pay struct...
6,6.0,0.9861,"learn, work, good, lot, technology, place, experience, thing, get, great",i be fortunate to start my career in tc i get to learn a lot and work across 7 different project...
7,7.0,0.9757,"train, provide, opportunity, employee, growth, skill, organization, technology, service, grow",tata consultancy service limit tc be an indian multinational information technology it service b...


### Sklearn Topic Modeling

In [52]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV 


In [53]:

lda = LatentDirichletAllocation(
                learning_method="batch",
                random_state=100,
                n_components=7,
                doc_topic_prior=0.01,
                topic_word_prior=0.31,
                n_jobs=-2
            )
lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.01,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=-2,
                          perp_tol=0.1, random_state=100, topic_word_prior=0.31,
                          total_samples=1000000.0, verbose=0)

In [54]:
def print_topics(model, vectorizer, top_n=50):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('•', 2152.3043183731656), ('development', 1730.0918149780796), ('work', 1473.2847012084758), ('system', 1457.0119540512298), ('use', 1425.1357544326597), ('application', 1391.1172856916562), ('support', 1212.6857484482955), ('software', 1169.3874339943602), ('test', 917.8518795823762), ('issue', 879.8888915237427), ('project', 874.5352259273773), ('server', 776.6539095713795), ('design', 766.4158022574471), ('network', 754.6627604125773), ('user', 744.0942142497814), ('client', 674.6770678451837), ('experience', 650.6240976933344), ('business', 598.0986940791116), ('implement', 588.7212301902206), ('&', 574.7780458279402), ('database', 557.4207393155214), ('management', 556.1743915970684), ('requirement', 556.0817790424478), ('process', 544.420606179033), ('report', 542.8321989617361), ('involve', 524.7963207415515), ('code', 489.04390144066673), ('service', 467.07465160550277), ('create', 463.18197149801205), ('developer', 441.6234514420656), ('manage', 429.87590

[('work', 10392.353177745086), ('great', 5370.733354160755), ('employee', 2715.459069728282), ('benefit', 2467.7090289603602), ('place', 2390.7375114337424), ('management', 2043.3726178089642), ('place work', 1536.6564625462163), ('environment', 1445.1097338282866), ('hour', 1349.2187289460333), ('balance', 1329.1694301086848), ('culture', 1224.420864097882), ('time', 1065.5473205088151), ('pay', 1042.8836855528596), ('great place', 1015.4643868322244), ('well', 1005.0291481525225), ('people', 997.9322165042648), ('life', 904.2763668001636), ("'s", 885.4578887642748), ('flexible', 882.3676588851167), ('good', 870.698314616632), ('great work', 858.3601608209251), ('opportunity', 833.5919583938993), ('care', 764.4202211100444), ('need', 763.6882229722637), ('home', 698.561504992571), ('work environment', 678.2519329026813), ('like', 659.0393635460014), ('make', 649.1010295495588), ('advancement', 595.5452098553545), ('one', 588.122356651096), ('excellent', 587.908569696159), ('lot', 572.

In [56]:
aspect = {}
for idx, topic in enumerate(lda.components_):
    aspect['Aspect {0}'.format(str(idx))] = [vectorizer.get_feature_names()[i]
                                                         for i in topic.argsort()[:-100 - 1:-1]]

with open("../results/Sklearn-LDA/sklearn-word-lda-topic-k-7-a-0.01-b-0.31.json", "w") as f:
    json.dump(aspect, f)

TypeError: list indices must be integers or slices, not str

In [ ]:
# Build a Non-Negative Matrix Factorization Model
nmf = NMF(n_components=7
                  ,init='nndsvd'
                 ,random_state=100,
                 alpha=0.61)

nmf.fit(X)


In [ ]:
print("NMF Model:")
print_topics(nmf, vectorizer)
print("=" * 20)

In [ ]:
spect = {}
for idx, topic in enumerate(nmf.components_):
    aspect['Aspect {0}'.format(str(idx))] = {vectorizer.get_feature_names()[i]
                                                         for i in topic.argsort()[:-100 - 1:-1]}

with open("../results/Sklearn-LDA/sklearn-word-nmf-topic-k-7-a-0.61.json", "w") as f:
    json.dump(aspect, f)